In [ ]:
import cv2
import numpy as np
import pandas as pd
from sklearn.model_selection import KFold
from sklearn import metrics
import keras
from keras.models import Model
from keras.optimizers import Adam
from keras.applications.inception_v3 import InceptionV3
from keras.layers import Dense, Input, Flatten, Dropout, GlobalAveragePooling2D
from tensorflow.keras.layers import BatchNormalization
from keras.callbacks import ModelCheckpoint, EarlyStopping, ReduceLROnPlateau
from google.colab import drive
drive.mount('/content/gdrive')

MessageError: Error: credential propagation was unsuccessful

In [ ]:
path = "/content/gdrive/My Drive/train/"
train_set = pd.read_csv('/content/gdrive/My Drive/train_labels.csv')
train_label = np.array(train_set['invasive'].iloc[: ])
train_files = []
for i in range(len(train_set)):
    train_files.append(path + str(int(train_set.iloc[i][0])) +'.jpg')
train_set['name'] = train_files
print(train_set.shape)
train_set.head()

(2295, 2)


,name,invasive
0,/content/gdrive/My Drive/train/1.jpg,0
1,/content/gdrive/My Drive/train/2.jpg,0
2,/content/gdrive/My Drive/train/3.jpg,1
3,/content/gdrive/My Drive/train/4.jpg,0
4,/content/gdrive/My Drive/train/5.jpg,1


In [ ]:
path = "/content/gdrive/My Drive/test/"
test_set = pd.read_csv('/content/gdrive/My Drive/sample_submission.csv')
test_files = []
for i in range(len(test_set)):
    test_files.append(path + str(int(test_set.iloc[i][0])) +'.jpg')
print(test_set.shape)
test_set.head()

(1531, 2)


,name,invasive
0,1,0.5
1,2,0.5
2,3,0.5
3,4,0.5
4,5,0.5


In [ ]:
img_dim = (800, 800, 3)
input = Input(shape=img_dim)
base_model = InceptionV3(include_top=False,weights='imagenet',input_shape=img_dim)
bn = BatchNormalization()(input)
x = base_model(bn)
x = GlobalAveragePooling2D()(x)
x = Dropout(0.5)(x)
output = Dense(1, activation='sigmoid')(x)
model = Model(input, output)
model.summary()

87910968/87910968 [==============================] - 0s 0us/step
Model: "model"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 input_1 (InputLayer)        [(None, 800, 800, 3)]     0         
                                                                 
 batch_normalization_94 (Ba  (None, 800, 800, 3)       12        
 tchNormalization)                                               
                                                                 
 inception_v3 (Functional)   (None, 23, 23, 2048)      21802784  
                                                                 
 global_average_pooling2d (  (None, 2048)              0         
 GlobalAveragePooling2D)                                         
                                                                 
 dropout (Dropout)           (None, 2048)              0         
                                                              

In [ ]:
def train_model(model, batch_size, epochs, img_size, x, y, test, n_fold, kf):
    roc_auc = metrics.roc_auc_score
    preds_train = np.zeros(len(x), dtype = float)
    preds_test = np.zeros(len(test), dtype = float)
    train_scores = []; valid_scores = []

    i = 1

    for train_index, test_index in kf.split(x):
        x_train = x.iloc[train_index]; x_valid = x.iloc[test_index]
        y_train = y[train_index]; y_valid = y[test_index]

        def augment(src, choice):
            if choice == 0:
                # Rotate 90
                src = np.rot90(src, 1)
            if choice == 1:
                # flip vertically
                src = np.flipud(src)
            if choice == 2:
                # Rotate 180
                src = np.rot90(src, 2)
            if choice == 3:
                # flip horizontally
                src = np.fliplr(src)
            if choice == 4:
                # Rotate 90 counter-clockwise
                src = np.rot90(src, 3)
            if choice == 5:
                # Rotate 180 and flip horizontally
                src = np.rot90(src, 2)
                src = np.fliplr(src)
            return src

        def train_generator():
            while True:
                for start in range(0, len(x_train), batch_size):
                    x_batch = []
                    y_batch = []
                    end = min(start + batch_size, len(x_train))
                    train_batch = x_train[start:end]
                    for filepath, tag in train_batch.values:
                        img = cv2.imread(filepath)
                        img = cv2.resize(img, img_size)
                        img = augment(img, np.random.randint(6))
                        x_batch.append(img)
                        y_batch.append(tag)
                    x_batch = np.array(x_batch, np.float32) / 255.
                    y_batch = np.array(y_batch, np.uint8)
                    yield x_batch, y_batch

        def valid_generator():
            while True:
                for start in range(0, len(x_valid), batch_size):
                    x_batch = []
                    y_batch = []
                    end = min(start + batch_size, len(x_valid))
                    valid_batch = x_valid[start:end]
                    for filepath, tag in valid_batch.values:
                        img = cv2.imread(filepath)
                        img = cv2.resize(img, img_size)
                        img = augment(img, np.random.randint(6))
                        x_batch.append(img)
                        y_batch.append(tag)
                    x_batch = np.array(x_batch, np.float32) / 255.
                    y_batch = np.array(y_batch, np.uint8)
                    yield x_batch, y_batch

        def test_generator():
            while True:
                for start in range(0, len(test), batch_size):
                    x_batch = []
                    end = min(start + batch_size, len(test))
                    test_batch = test[start:end]
                    for filepath in test_batch:
                        img = cv2.imread(filepath)
                        img = cv2.resize(img, img_size)
                        x_batch.append(img)
                    x_batch = np.array(x_batch, np.float32) / 255.
                    yield x_batch

        callbacks = [EarlyStopping(monitor='val_loss', patience=3, verbose=1, min_delta=1e-4),
             ReduceLROnPlateau(monitor='val_loss', factor=0.1, patience=1, cooldown=1,
                               verbose=1, min_lr=1e-7),
             ModelCheckpoint(filepath='inception.fold_' + str(i) + '.hdf5', verbose=1,
                             save_best_only=True, save_weights_only=True, mode='auto')]

        train_steps = len(x_train) / batch_size
        valid_steps = len(x_valid) / batch_size
        test_steps = len(test) / batch_size

        model = model

        model.compile(optimizer=Adam(lr=1e-4), loss='binary_crossentropy',
                      metrics = ['accuracy'])

        model.fit(train_generator(), train_steps, epochs=epochs, verbose=1,
                            callbacks=callbacks, validation_data=valid_generator(),
                            validation_steps=valid_steps)

        model.load_weights(filepath='inception.fold_' + str(i) + '.hdf5')

        print('Running validation predictions on fold {}'.format(i))
        preds_valid = model.predict_generator(generator=valid_generator(),
                                      steps=valid_steps, verbose=1)[:, 0]

        print('Running train predictions on fold {}'.format(i))
        preds_train = model.predict_generator(generator=train_generator(),
                                      steps=train_steps, verbose=1)[:, 0]

        valid_score = roc_auc(y_valid, preds_valid)
        train_score = roc_auc(y_train, preds_train)
        print('Val Score:{} for fold {}'.format(valid_score, i))
        print('Train Score: {} for fold {}'.format(train_score, i))

        valid_scores.append(valid_score)
        train_scores.append(train_score)
        print('Avg Train Score:{0:0.5f}, Val Score:{1:0.5f} after {2:0.5f} folds'.format
              (np.mean(train_scores), np.mean(valid_scores), i))

        print('Running test predictions with fold {}'.format(i))

        preds_test_fold = model.predict(generator=test_generator(),steps=test_steps, verbose=1)[:, -1]

        preds_test += preds_test_fold

        print('\n\n')

        i += 1

        if i <= n_fold:
            print('Now beginning training for fold {}\n\n'.format(i))
        else:
            print('Finished training!')

    preds_test /= n_fold


    return preds_test

In [ ]:
batch_size = 5
epochs = 50
n_fold = 5
img_size = (800, 800)
kf = KFold(n_splits=n_fold, shuffle=True)

test_pred = train_model(model, batch_size, epochs, img_size, train_set,
                        train_label, test_files, n_fold, kf)

test_set['invasive'] = test_pred
test_set.to_csv('/content/gdrive/My Drive/submission.csv', index = None)

<ipython-input-7-cde58208ef5a>:3: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  preds_train = np.zeros(len(x), dtype = np.float)
<ipython-input-7-cde58208ef5a>:4: DeprecationWarning: `np.float` is a deprecated alias for the builtin `float`. To silence this warning, use `float` by itself. Doing this will not modify any behavior and is safe. If you specifically wanted the numpy scalar type, use `np.float64` here.
Deprecated in NumPy 1.20; for more details and guidance: https://numpy.org/devdocs/release/1.20.0-notes.html#deprecations
  preds_test = np.zeros(len(test), dtype = np.float)
<ipython-input-7-cde58208ef5a>:97: UserWarning: `Model.fit_generator` is depreca

Epoch 1/50
368/367 [==============================] - ETA: -1s - loss: 0.4056 - accuracy: 0.8246
Epoch 1: val_loss improved from inf to 0.63609, saving model to inception.fold_1.hdf5
367/367 [==============================] - 957s 2s/step - loss: 0.4056 - accuracy: 0.8246 - val_loss: 0.6361 - val_accuracy: 0.8257 - lr: 0.0010
Epoch 2/50
368/367 [==============================] - ETA: 0s - loss: 0.3539 - accuracy: 0.8584
Epoch 2: val_loss improved from 0.63609 to 0.30848, saving model to inception.fold_1.hdf5
367/367 [==============================] - 228s 620ms/step - loss: 0.3539 - accuracy: 0.8584 - val_loss: 0.3085 - val_accuracy: 0.8736 - lr: 0.0010
Epoch 3/50
368/367 [==============================] - ETA: 0s - loss: 0.3074 - accuracy: 0.8742
Epoch 3: val_loss improved from 0.30848 to 0.19822, saving model to inception.fold_1.hdf5
367/367 [==============================] - 224s 609ms/step - loss: 0.3074 - accuracy: 0.8742 - val_loss: 0.1982 - val_accuracy: 0.9259 - lr: 0.0010
Epoc

<ipython-input-7-cde58208ef5a>:104: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  preds_valid = model.predict_generator(generator=valid_generator(),


91/91 [==============================] - 29s 310ms/step
Running train predictions on fold 1


<ipython-input-7-cde58208ef5a>:108: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  preds_train = model.predict_generator(generator=train_generator(),


...

<ipython-input-7-cde58208ef5a>:123: UserWarning: `Model.predict_generator` is deprecated and will be removed in a future version. Please use `Model.predict`, which supports generators.
  preds_test_fold = model.predict_generator(generator=test_generator(),


306/306 [==============================] - 88s 282ms/step





Now beginning training for fold 5




<ipython-input-7-cde58208ef5a>:97: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  model.fit_generator(train_generator(), train_steps, epochs=epochs, verbose=1,


Epoch 1/50
368/367 [==============================] - ETA: 0s - loss: 0.2187 - accuracy: 0.9325
Epoch 1: val_loss improved from inf to 0.39901, saving model to inception.fold_5.hdf5
367/367 [==============================] - 267s 644ms/step - loss: 0.2187 - accuracy: 0.9325 - val_loss: 0.3990 - val_accuracy: 0.8257 - lr: 0.0010
Epoch 2/50
368/367 [==============================] - ETA: 0s - loss: 0.2147 - accuracy: 0.9254
Epoch 2: val_loss improved from 0.39901 to 0.20515, saving model to inception.fold_5.hdf5
367/367 [==============================] - 220s 598ms/step - loss: 0.2147 - accuracy: 0.9254 - val_loss: 0.2052 - val_accuracy: 0.9216 - lr: 0.0010
Epoch 3/50
368/367 [==============================] - ETA: 0s - loss: 0.1963 - accuracy: 0.9325
Epoch 3: ReduceLROnPlateau reducing learning rate to 0.00010000000474974513.

Epoch 3: val_loss did not improve from 0.20515
367/367 [==============================] - 219s 595ms/step - loss: 0.1963 - accuracy: 0.9325 - val_loss: 0.2235 - v